In [1]:
pip install pandas numpy matplotlib seaborn scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_curve, auc, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage


In [3]:
# Load your dataset
data = pd.read_csv('/kaggle/input/house-health-2/processed_04.csv')

# Drop non-numeric columns if not needed for clustering
features = data[['x', 'y', 'z']]
labels = data['label']

# Normalize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
# Perform hierarchical clustering
Z = linkage(scaled_features, 'ward')

# Plot the dendrogram
plt.figure(figsize=(10, 7))
dendrogram(Z, truncate_mode='lastp', p=50, show_leaf_counts=True)
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('Sample index')
plt.ylabel('Distance')
plt.show()

In [ ]:
# Split the data
X_train, X_test, y_train, y_test = train_test_split(scaled_features, labels, test_size=0.3, random_state=42)

# Train Naive Bayes classifier
model = GaussianNB()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

# ROC Curve
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Class 0', 'Class 1'],
            yticklabels=['Class 0', 'Class 1'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# Positive and Negative Outcomes
results = pd.DataFrame({'True Label': y_test, 'Predicted Label': y_pred, 'Probability': y_prob})
outcomes = results.groupby(['True Label', 'Predicted Label']).size().unstack()

plt.figure(figsize=(10, 7))
outcomes.plot(kind='bar', stacked=True)
plt.title('Positive/Negative Outcomes')
plt.xlabel('True Label')
plt.ylabel('Count')
plt.show()